In [2]:
import numpy as np
#generate the mapping file of local loci and general locis
# matrix = np.loadtxt("../GM12878/chr21_5kb/chr21_5000.dist")
dir="/home/yangyi/study/biye"
mappingdir="/data1/ghy_data/IMR90/3DResults/NeRV-3D-DV/chr20_5kb/0.2/2Mb/"
matrix = np.loadtxt("/data1/ghy_data/IMR90/3DResults/NeRV-3D-DV/chr20_5kb/0.2/chr20_5000_0.2.dist")
j = 0
data_index=[]
with open(mappingdir+"mapping.txt", "w") as out:
    for i in range(len(matrix)):
        if not (matrix[i] == 0).all():
            out.write("\t".join((str(i),str(j))) + "\n")
            data_index.append(i)
            j = j + 1
        else:
            out.write("\t".join((str(i),'nan')) + "\n")
l = len(data_index)
data = np.zeros((l,l))
for i in range(l):
    for j in range(l):
        data[i,j] = matrix[data_index[i],data_index[j]]

#np.savetxt(mappingdir+"data.txt",data)
data = np.loadtxt("/data1/ghy_data/IMR90/3DResults/NeRV-3D-DV/chr20_5kb/0.2/data.txt")

KeyboardInterrupt: 

In [5]:
# overlap = np.zeros((int(len(data)/10+1),int(len(data)/10+1)))
# for i in range(0,len(data),10):
#     for j in range(0,len(data),10):
#         overlap[int(i/10),int(j/10)] = np.nanmean(data[i:i+10,j:j+10])
#generate the global distance file in low reslotion
mappingdir="/data1/ghy_data/IMR90/3DResults/NeRV-3D-DV/chr20_5kb/0.2/2Mb/"
data = np.loadtxt("/data1/ghy_data/IMR90/3DResults/NeRV-3D-DV/chr20_5kb/0.2/data.txt")
overlap=data[::20,::20]
file = mappingdir+"genernal.txt"
with open(file, "w") as out:
    out.write(str(len(overlap)) + "\n")
    for i in range(len(overlap)):
        out.write("\t".join(str(item) for item in overlap[i]) + "\n")
    out.close()

KeyboardInterrupt: 

In [6]:
import math
def get_tads(id):

    # 域内的点,每个子域有100个点
    start = id *400
    end = (id + 1) *400

    if end > len(data):
        end = len(data)
    # 用于输入的文件
    file =mappingdir+ "tad{}.txt".format(id)
    with open(file, "w") as out:
        out.write(str(end - start) + "\n")
        for i in range(start,end):
            out.write("\t".join(str(item) for item in data[i,start:end]) + "\n")
        out.close()

for i in range(math.ceil(len(data)/400)):
    get_tads(i)

In [20]:
import numpy as np
import math
from scipy.spatial.transform import Rotation as R
#执行之前，需要test.sh,用NeRV-3D计算每个TADs结构
genernal = np.loadtxt(mappingdir+"genernal.xyz", skiprows=1)
with open("res.xyz", "w") as out:
    for i in range(math.ceil(len(genernal) / 10)):
        file = "../IMR90/chr22_5kb/structure/strtad{}.xyz".format(i)
        tad = np.loadtxt(file, skiprows=1)
        inferredLow = np.zeros((math.ceil(len(tad) / 10),3))
        for j in range(math.ceil(len(tad) / 10)):
            inferredLow[j] = np.mean(tad[j*10:j*10+10,])
        trueLow = genernal[i*10:i*10+10,]

        # center
        trueLow = trueLow - np.mean(trueLow, axis=0)
        inferredLow = inferredLow - np.mean(inferredLow, axis=0)

        #rescale
        true_sum = sum([np.sqrt(np.sum(coord**2)) for coord in trueLow])
        inferred_sum = sum([np.sqrt(np.sum(coord**2)) for coord in inferredLow])
        scaling_factor = true_sum / inferred_sum
        inferredLow = inferredLow * scaling_factor

        # rotate
        estimated_rotation, rmsd = R.align_vectors(trueLow,inferredLow)
        a_transformed = estimated_rotation.apply(tad)
        a_transformed = a_transformed - np.mean(a_transformed, axis=0)
        true_sum = np.mean([np.sqrt(np.sum(coord**2)) for coord in trueLow])
        inferred_sum = np.mean([np.sqrt(np.sum(coord**2)) for coord in a_transformed])
        scaling_factor = true_sum / inferred_sum
        a_transformed = a_transformed * scaling_factor
        a_transformed = a_transformed + np.mean(genernal[i*10:i*10+10,], axis=0)
        for j in range(len(a_transformed)):
            out.write("\t".join(str(item) for item in a_transformed[j]) + "\n")
    out.close()

/home/yangyi/anaconda3/envs/biye/lib/python3.6/site-packages/scipy/spatial/transform/rotation.py:1967: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  warnings.warn("Optimal rotation is not uniquely or poorly defined "


In [2]:
import numpy as np
map = np.loadtxt("/data1/ghy_data/GM12878/3DResults/NeRV-3D-DV/chr19_5kb/mapping.txt")
xyz = np.loadtxt("/data1/ghy_data/GM12878/3DResults/NeRV-3D-DV/chr19_5kb/highres.xyz")

with open("/data1/ghy_data/GM12878/3DResults/NeRV-3D-DV/chr19_5kb/highresplusnan.xyz", "w") as out:
    for i in range(len(map)):
        if np.isnan(map[i,1]):
            out.write("\t".join(("nan", "nan", "nan")) + "\n")
        else:
            out.write("\t".join(str(item) for item in xyz[int(map[i,1])]) + "\n")


IndexError: index 11100 is out of bounds for axis 0 with size 11100

In [19]:
import numpy as np

xyz = np.loadtxt("restad.xyz")
# xyz = np.loadtxt("../IMR90/chr22_5kb/structure/genernal.xyz", skiprows=1)
res = 25
tad = np.loadtxt("../GM12878/chr21_5kb/tads.txt", dtype=int)


outpath = "test.xyz"
with open(outpath, "w") as out:
    for i in range(len(tad)):
        start = int( tad[i,1] / res / 1000 )
        end = int( tad[i,3] / res / 1000 )
        for j in range(start,end+1):
            if j<len(xyz) and not np.isnan(xyz[j,0]):
                out.write("[{},{},{},{}],".format(str(xyz[j,0]),str(xyz[j,1]),str(xyz[j,2]),str(i)) + "\n")
    out.close()